<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/char_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
lines = []
for line in f:
  line = line.strip() # \r, \n 제거
  line = line.lower()
  line = line.decode('ascii', 'ignore') # 아스키 문자만 남기고 삭제
  if len(line) > 0:
    lines.append(line)

f.close()

In [5]:
print(lines[:5])
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수 : %d' % len(text))
print(text[:200])

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll', 'this ebook is for the use of anyone anywhere in the united states and', 'most other parts of the world at no cost and with almost no restrictions', 'whatsoever. you may copy it, give it away or re-use it under the terms', 'of the project gutenberg license included with this ebook or online at']
문자열의 길이 또는 총 글자의 개수 : 159484
the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


In [6]:
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)
print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 56


In [7]:
char_to_index = dict((c, i) for (i, c) in enumerate(char_vocab))
print(char_to_index)

index_to_char = dict((i, c) for (c, i) in char_to_index.items())
print(index_to_char)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '/', 14: '0', 15: '1', 16: '2', 17: '3', 18: '4', 19: '5', 20: '6', 21: '7', 22: '8', 23: '9', 24: ':', 25: ';', 26: '?', 27: '[', 28: ']', 29: '_', 30: 'a', 31: 'b', 32: 'c', 33: 'd', 34: 'e', 35: 'f', 36: 'g', 37: 'h', 38: 'i', 39: 'j', 40: 'k', 41: 'l', 42: 'm', 43: 'n', 44: 'o', 45: 'p', 46: 'q', 47: 'r', 48: 's', 49: 't', 50: 'u', 51: 'v', 52: 'w', 53: 'x', 54: 'y', 55: 'z'}


In [8]:
seq_length = 60 # 문장의 길이를 60으로 설정
n_samples = int(np.floor((len(text) - 1) / seq_length)) # y_train을 오른쪽으로 한번 shift 해야 하므로 1을 빼줌
print('문장 샘플의 수 {}'.format(n_samples))

문장 샘플의 수 2658


In [9]:
x_train = list()
y_train = list()

for i in range(n_samples):
  x_sample  = text[i * seq_length : (i + 1) * seq_length]
  x_encoded = [char_to_index[ch] for ch in x_sample]
  x_train.append(x_encoded)

  y_sample = text[i * seq_length + 1 : (i + 1) * seq_length + 1]
  y_encoded = [char_to_index[ch] for ch in y_sample]
  y_train.append(y_encoded)

print(x_train[0])
print(y_train[0])
print(x_train[1])
print(y_train[1])
print(np.array(x_train).shape)

[49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
[37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]
[43, 33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54]
[33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54, 52]
(2658, 60)


In [10]:
# 글자 단위 RNN에서는 입력 시퀀스에 대해 워드 임베딩을 하지 않으므로 입력 시퀀스인 x_train에 대해서도 원-핫 인코딩 수행

x_train = to_categorical(x_train) # one-hot encoding
y_train = to_categorical(y_train)

print('x_train의 크기(shape) : {}'.format(x_train.shape))
print('y_train의 크기(shape) : {}'.format(y_train.shape))
# batch_size(num_of_samples) = 2658, timesteps(input_length) = 60, input_dim(vocab_size) = 56)

x_train의 크기(shape) : (2658, 60, 56)
y_train의 크기(shape) : (2658, 60, 56)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

model = Sequential()
# input_sequence : (2658, 60, 56)
model.add(LSTM(256, input_shape=(None, x_train.shape[2]), return_sequences=True))
# (2658, 60, 56) --LSTM-> (2658, 60, 256)
model.add(LSTM(256, return_sequences=True))
# (2658, 60, 256) --LSTM-> (2658, 60, 256)
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
# (2658, 60, 256) --softmax-> (2658, 60, 56)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=80, verbose=2)

84/84 - 41s - loss: 0.7682 - accuracy: 0.7718
Epoch 70/80
84/84 - 41s - loss: 0.7500 - accuracy: 0.7773
Epoch 71/80
84/84 - 40s - loss: 0.7246 - accuracy: 0.7858
Epoch 72/80
84/84 - 41s - loss: 0.7102 - accuracy: 0.7904
Epoch 73/80
84/84 - 41s - loss: 0.6933 - accuracy: 0.7959
Epoch 74/80
84/84 - 41s - loss: 0.6737 - accuracy: 0.8024
Epoch 75/80
84/84 - 41s - loss: 0.6625 - accuracy: 0.8048
Epoch 76/80
84/84 - 41s - loss: 0.6646 - accuracy: 0.8011
Epoch 77/80
84/84 - 42s - loss: 0.6248 - accuracy: 0.8180
Epoch 78/80
84/84 - 42s - loss: 0.6087 - accuracy: 0.8235
Epoch 79/80
84/84 - 42s - loss: 0.6093 - accuracy: 0.8204
Epoch 80/80
84/84 - 41s - loss: 0.5815 - accuracy: 0.8311


In [50]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 인덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size)) # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in range(length):
        X[0][i][ix[-1]] = 1 # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [67]:
rm -r drive

In [60]:
sentence_generation(model, 100)

30 번 글자 a 로 예측을 시작!
and now alice to herself in a tone of great collectee to herself, and the mock turtle in the directi

'and now alice to herself in a tone of great collectee to herself, and the mock turtle in the directio'

In [68]:
model.save('./drive/MyDrive/models/LSTM_by_char')

INFO:tensorflow:Assets written to: ./drive/MyDrive/models/LSTM_by_char/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/models/LSTM_by_char/assets
